# 0. 필요한 라이브러리 설치

In [ ]:
!pip install pycaret

     |████████████████████████████████| 301 kB 7.4 MB/s 
     |████████████████████████████████| 86 kB 4.4 MB/s 
     |████████████████████████████████| 6.8 MB 14.5 MB/s 
     |████████████████████████████████| 114 kB 42.4 MB/s 
     |████████████████████████████████| 261 kB 18.4 MB/s 
     |████████████████████████████████| 167 kB 8.6 MB/s 
     |████████████████████████████████| 15.6 MB 34.6 MB/s 
     |████████████████████████████████| 56 kB 3.1 MB/s 
     |████████████████████████████████| 1.3 MB 45.1 MB/s 
     |████████████████████████████████| 1.7 MB 29.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 274 kB 49.1 MB/s 
     |████████████████████████████████| 2.0 MB 41.0 MB/s 
     |████████████████████████████████| 675 kB 46.9 MB/s 
     |████████████████████████████████| 102 kB 10.9 MB/s 
     |██████████████████

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  cuda-command-line-tools-10-0 cuda-command-line-tools-10-1
  cuda-command-line-tools-11-0 cuda-compiler-10-0 cuda-compiler-10-1
  cuda-compiler-11-0 cuda-cuobjdump-10-0 cuda-cuobjdump-10-1
  cuda-cuobjdump-11-0 cuda-cupti-10-0 cuda-cupti-10-1 cuda-cupti-11-0
  cuda-cupti-dev-11-0 cuda-documentation-10-0 cuda-documentation-10-1
  cuda-documentation-11-0 cuda-documentation-11-1 cuda-gdb-10-0 cuda-gdb-10-1
  cuda-gdb-11-0 cuda-gpu-library-advisor-10-0 cuda-gpu-library-advisor-10-1
  cuda-libraries-10-0 cuda-libraries-10-1 cuda-libraries-11-0
  cuda-memcheck-10-0 cuda-memcheck-10-1 cuda-memcheck-11-0 cuda-nsight-10-0
  cuda-nsight-10-1 cuda-nsight-11-0 cuda-nsight-11-1 cuda-nsight-compute-10-0
  cuda-nsight-compute-10-1 cuda-nsight-compute-11-0 cuda-nsight-compute-11-1
  cuda-nsight-systems-10-1 cuda-nsight-systems-

In [ ]:
pip install numpy --upgrade

     |████████████████████████████████| 15.7 MB 9.8 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
yellowbrick 1.3.post1 requires numpy<1.20,>=1.16.0, but you have numpy 1.21.5 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


런타임 다시 시작

# 1. 경로 설정 및 필요한 라이브러리 로드

In [ ]:
# 경로 설정
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm

#모델 임포트
import pycaret 
pycaret.__version__
from pycaret.regression import *

#from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import HuberRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# 시각화 모듈 임포트
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
from matplotlib import font_manager, rc

%matplotlib inline
plt.rcParams['axes.unicode_minus'] = False
plt.rc('font', family = 'NanumBarunGothic')
plt.style.use('ggplot')
%config InlineBackend.figure_format = 'retina'

# 경로를 입력해주세요.
path = '/content/drive/MyDrive/조달공모전/data/'

Mounted at /content/drive


/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


# 2. 모델링

변수 제거하기

In [ ]:
df = pd.read_csv(path+'cleaned_data.csv')
df.head(n=1)

,수요기관구분,수요기관지역명,긴급공고여부,추정가격,배정예산,기초금액,예정가격,참가수,업종제한여부,공동도급구성방식명,낙찰하한율,입찰기간,정정공고여부,취소공고여부,재입찰허용여부,공동도급협정서접수방식,낙찰자결정여부,낙찰업체투찰금액,국제입찰여부,낙찰자결정적용법규,지역제한여부
0,지방자치단체,경상남도,N,17.188901,17.284211,17.284211,17.283283,8.0,Y,분담이행,88.0,2,Y,N,Y,협정서전자제출,Y,17.1584,N,지방계약법,Y


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30703 entries, 0 to 30702
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   수요기관구분       30703 non-null  object 
 1   수요기관지역명      30703 non-null  object 
 2   긴급공고여부       30703 non-null  object 
 3   추정가격         30703 non-null  float64
 4   배정예산         30703 non-null  float64
 5   기초금액         30703 non-null  float64
 6   예정가격         30703 non-null  float64
 7   참가수          30703 non-null  float64
 8   업종제한여부       30703 non-null  object 
 9   공동도급구성방식명    30703 non-null  object 
 10  낙찰하한율        30703 non-null  float64
 11  입찰기간         30703 non-null  int64  
 12  정정공고여부       30703 non-null  object 
 13  취소공고여부       30703 non-null  object 
 14  재입찰허용여부      30703 non-null  object 
 15  공동도급협정서접수방식  30703 non-null  object 
 16  낙찰자결정여부      30703 non-null  object 
 17  낙찰업체투찰금액     30703 non-null  float64
 18  국제입찰여부       30703 non-null  object 
 19  낙찰자결

In [ ]:
df.columns

Index(['수요기관구분', '수요기관지역명', '긴급공고여부', '추정가격', '배정예산', '기초금액', '예정가격', '참가수',
       '업종제한여부', '공동도급구성방식명', '낙찰하한율', '입찰기간', '정정공고여부', '취소공고여부', '재입찰허용여부',
       '공동도급협정서접수방식', '낙찰자결정여부', '낙찰업체투찰금액', '국제입찰여부', '낙찰자결정적용법규', '지역제한여부'],
      dtype='object')

3개의 모델에서 feature importance를 확인한 후 2개의 모델 이상에서 나타난 변수들로 feature importance가 높은 X 변수 4개를 추림

'기초금액',  '수요기관구분', '입찰기간', '수요기관지역명', '배정예산', '공동도급구성방식명'

In [ ]:
#df_new = df[['기초금액', '배정예산', '수요기관지역명', '공동도급협정서접수방식', '수요기관구분', '공동도급구성방식명', '입찰기간','낙찰업체투찰금액']]
#df_new = df[['기초금액', '배정예산', '수요기관구분', '공동도급구성방식명', '입찰기간','낙찰업체투찰금액']]
df_new = df[['기초금액',  '수요기관구분', '입찰기간', 
             '수요기관지역명', '배정예산', '공동도급구성방식명',
             '낙찰업체투찰금액']]
df_new.columns

Index(['기초금액', '수요기관구분', '입찰기간', '수요기관지역명', '배정예산', '공동도급구성방식명', '낙찰업체투찰금액'], dtype='object')

In [ ]:
newdf = pd.get_dummies(df_new)
newdf.head()

,기초금액,입찰기간,배정예산,낙찰업체투찰금액,수요기관구분_공기업,수요기관구분_교육기관,수요기관구분_국가기관,수요기관구분_기타공공기관,수요기관구분_기타기관,수요기관구분_준정부기관,수요기관구분_지방공기업,수요기관구분_지방자치단체,수요기관구분_지자체 출자출연기관,수요기관지역명_강원도,수요기관지역명_경기도,수요기관지역명_경상남도,수요기관지역명_경상북도,수요기관지역명_광주광역시,수요기관지역명_대구광역시,수요기관지역명_대전광역시,수요기관지역명_부산광역시,수요기관지역명_서울특별시,수요기관지역명_세종특별자치시,수요기관지역명_울산광역시,수요기관지역명_인천광역시,수요기관지역명_전라남도,수요기관지역명_전라북도,수요기관지역명_제주특별자치도,수요기관지역명_충청남도,수요기관지역명_충청북도,공동도급구성방식명_공동+분담이행,공동도급구성방식명_공동수급불허,공동도급구성방식명_공동이행,공동도급구성방식명_분담이행
0,17.284211,2,17.284211,17.158400,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,17.992335,6,17.992335,17.858562,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
2,17.981214,3,17.981214,17.852579,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
3,18.469357,3,18.469357,18.335401,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
4,17.670857,2,17.670857,17.550562,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0


In [ ]:
X = newdf.drop(['낙찰업체투찰금액'], axis=1)
y = newdf['낙찰업체투찰금액']

In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30703 entries, 0 to 30702
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   기초금액               30703 non-null  float64
 1   입찰기간               30703 non-null  int64  
 2   배정예산               30703 non-null  float64
 3   수요기관구분_공기업         30703 non-null  uint8  
 4   수요기관구분_교육기관        30703 non-null  uint8  
 5   수요기관구분_국가기관        30703 non-null  uint8  
 6   수요기관구분_기타공공기관      30703 non-null  uint8  
 7   수요기관구분_기타기관        30703 non-null  uint8  
 8   수요기관구분_준정부기관       30703 non-null  uint8  
 9   수요기관구분_지방공기업       30703 non-null  uint8  
 10  수요기관구분_지방자치단체      30703 non-null  uint8  
 11  수요기관구분_지자체 출자출연기관  30703 non-null  uint8  
 12  수요기관지역명_강원도        30703 non-null  uint8  
 13  수요기관지역명_경기도        30703 non-null  uint8  
 14  수요기관지역명_경상남도       30703 non-null  uint8  
 15  수요기관지역명_경상북도       30703 non-null  uint8  
 16  수요기관지역명_광주광역시      307

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)

In [ ]:
print('X train shape: ', X_train.shape)
print('y train shape: ', y_train.shape)
print('X test shape: ', X_test.shape)
print('y test shape: ', y_test.shape)

X train shape:  (24562, 33)
y train shape:  (24562,)
X test shape:  (6141, 33)
y test shape:  (6141,)


### Decision Tree Regressor

In [ ]:
dt_model = DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=2719, splitter='best')

In [ ]:
dt_model.fit(X_train, y_train)

dt_preds = dt_model.predict(X_test)

dt_rmse = np.sqrt(mean_squared_error(y_test, dt_preds))

print('RMSE: %f' % (dt_rmse))
print('r2 score: %f' % (r2_score(y_test, dt_preds)))

RMSE: 0.013670
r2 score: 0.999859


In [ ]:
dt_check = pd.DataFrame({'log_낙찰업체투찰금액' : y_test,
              'log_낙찰업체투찰금액_dt_pre' : dt_preds})

dt_check['낙찰업체투찰금액'] = np.expm1(dt_check['log_낙찰업체투찰금액'])
dt_check['낙찰업체투찰금액_dt_pre'] = np.expm1(dt_check['log_낙찰업체투찰금액_dt_pre'])

dt_check

,log_낙찰업체투찰금액,log_낙찰업체투찰금액_dt_pre,낙찰업체투찰금액,낙찰업체투찰금액_dt_pre
2105,18.089168,18.077818,71783700.00,70973581.00
24597,17.741289,17.731785,50692500.00,50213000.00
25929,17.217724,17.226922,30030490.00,30308000.00
20315,17.531242,17.527520,41088638.00,40936000.00
6268,17.894608,17.897594,59092140.00,59268800.00
...,...,...,...,...
21526,17.136572,17.134540,27689730.00,27633518.00
29446,3.730741,4.130676,40.71,61.22
12703,17.187778,17.195621,29144530.00,29374000.00
7987,17.785010,17.794948,52958000.00,53486952.00


In [ ]:
np.sqrt(mean_squared_error(dt_check['낙찰업체투찰금액'], dt_check['낙찰업체투찰금액_dt_pre']))

507160.85244983603

### huber regressor

In [ ]:
huber_model = HuberRegressor(alpha=0.0001, epsilon=1.35, fit_intercept=True, max_iter=100,
               tol=1e-05, warm_start=False)

In [ ]:
huber_model.fit(X_train, y_train)

huber_preds = huber_model.predict(X_test)

huber_rmse = np.sqrt(mean_squared_error(y_test, huber_preds))

print('RMSE: %f' % (huber_rmse))
print('r2 score: %f' % (r2_score(y_test, huber_preds)))

RMSE: 0.007913
r2 score: 0.999953


In [ ]:
huber_check = pd.DataFrame({'log_낙찰업체투찰금액' : y_test,
              'log_낙찰업체투찰금액_huber_pre' : huber_preds})

huber_check['낙찰업체투찰금액'] = np.expm1(huber_check['log_낙찰업체투찰금액'])
huber_check['낙찰업체투찰금액_huber_pre'] = np.expm1(huber_check['log_낙찰업체투찰금액_huber_pre'])

huber_check

,log_낙찰업체투찰금액,log_낙찰업체투찰금액_huber_pre,낙찰업체투찰금액,낙찰업체투찰금액_huber_pre
2105,18.089168,18.080272,71783700.00,7.114793e+07
24597,17.741289,17.742734,50692500.00,5.076582e+07
25929,17.217724,17.215402,30030490.00,2.996085e+07
20315,17.531242,17.530131,41088638.00,4.104300e+07
6268,17.894608,17.901096,59092140.00,5.947674e+07
...,...,...,...,...
21526,17.136572,17.143317,27689730.00,2.787712e+07
29446,3.730741,3.723008,40.71,4.038872e+01
12703,17.187778,17.197635,29144530.00,2.943324e+07
7987,17.785010,17.792521,52958000.00,5.335731e+07


In [ ]:
np.sqrt(mean_squared_error(huber_check['낙찰업체투찰금액'], huber_check['낙찰업체투찰금액_huber_pre']))

383850.3301804555

### RF(Random Forest)

In [ ]:
rf_model = RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=False,
                      random_state=2719, verbose=0, warm_start=False)

In [ ]:
rf_model.fit(X_train, y_train)

rf_preds = rf_model.predict(X_test)

rf_rmse = np.sqrt(mean_squared_error(y_test, rf_preds))

print('RMSE: %f' % (rf_rmse))
print('r2 score: %f' % (r2_score(y_test, rf_preds)))

RMSE: 0.046183
r2 score: 0.998388


In [ ]:
rf_check = pd.DataFrame({'log_낙찰업체투찰금액' : y_test,
              'log_낙찰업체투찰금액_rf_pre' : rf_preds})

rf_check['낙찰업체투찰금액'] = np.expm1(rf_check['log_낙찰업체투찰금액'])
rf_check['낙찰업체투찰금액_rf_pre'] = np.expm1(rf_check['log_낙찰업체투찰금액_rf_pre'])

rf_check

,log_낙찰업체투찰금액,log_낙찰업체투찰금액_rf_pre,낙찰업체투찰금액,낙찰업체투찰금액_rf_pre
2105,18.089168,18.080413,71783700.00,7.115796e+07
24597,17.741289,17.736864,50692500.00,5.046869e+07
25929,17.217724,17.219569,30030490.00,3.008596e+07
20315,17.531242,17.528969,41088638.00,4.099536e+07
6268,17.894608,17.899557,59092140.00,5.938528e+07
...,...,...,...,...
21526,17.136572,17.140218,27689730.00,2.779085e+07
29446,3.730741,4.147343,40.71,6.226568e+01
12703,17.187778,17.195317,29144530.00,2.936508e+07
7987,17.785010,17.795680,52958000.00,5.352613e+07


In [ ]:
np.sqrt(mean_squared_error(rf_check['낙찰업체투찰금액'], rf_check['낙찰업체투찰금액_rf_pre']))

388264.24113088194

## final model

In [ ]:
final_check = pd.DataFrame({'log_낙찰업체투찰금액' : y_test,
              'log_낙찰업체투찰금액_pre' : ((dt_preds + huber_preds + rf_preds) / 3)})

final_check['낙찰업체투찰금액'] = np.expm1(final_check['log_낙찰업체투찰금액'])
final_check['낙찰업체투찰금액_pre'] = np.expm1(final_check['log_낙찰업체투찰금액_pre'])

final_check

,log_낙찰업체투찰금액,log_낙찰업체투찰금액_pre,낙찰업체투찰금액,낙찰업체투찰금액_pre
2105,18.089168,18.079501,71783700.00,7.109311e+07
24597,17.741289,17.737127,50692500.00,5.048200e+07
25929,17.217724,17.220631,30030490.00,3.011793e+07
20315,17.531242,17.528874,41088638.00,4.099143e+07
6268,17.894608,17.899415,59092140.00,5.937688e+07
...,...,...,...,...
21526,17.136572,17.139358,27689730.00,2.776698e+07
29446,3.730741,4.000343,40.71,5.361686e+01
12703,17.187778,17.196191,29144530.00,2.939076e+07
7987,17.785010,17.794383,52958000.00,5.345675e+07


In [ ]:
np.sqrt(mean_squared_error(final_check['낙찰업체투찰금액'], final_check['낙찰업체투찰금액_pre']))

387671.04114187387

In [ ]:
print('r2 score: %f' % (r2_score(y_test, final_check['log_낙찰업체투찰금액_pre'])))

r2 score: 0.999766


# 4. 저장

In [ ]:
final_check.to_csv(path + "service_preds.csv", index = False)